In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

SALARIES_AND_WAGES_FILENAME = "salaries_and_wages_df.xlsx"
CPI_FILENAME = "consumers-price-index-june-2024-quarter-seasonally-adjusted.csv"

In [2]:
salaries_and_wages = pd.read_excel(f'./data/{SALARIES_AND_WAGES_FILENAME}')
cpi = pd.read_csv(f'./data/{CPI_FILENAME}')

In [3]:
# Of Interest is understanding the relationship between salaries and wages data and the consumer price index data
# Can we build a model to predict the CPI based on the salaries and wages data?

In [4]:
# Considerations:

# The model will be a regression model, as we are using numerical data to predict a numerical value

# We're dealing with timeseries data, so when separating data into training / testing sets
# we won't shuffle the data or randomise it, instead we will partition the datasets by date.

# The CPI dataset is seasonally adjusted-
# so we will use the seasonally adjusted salaries and wages data
